<h1>SIT307 Group Assignment 1 notebook</h1>

<h3>Group 3</h3>
<p>By:<br>
    Aaron Norwood,218330434<br>
    Joshua Anthony, 219466473<br>
    Roger Middenway, 217602784<br>
    David Adams, 216110104<br>
    Linden Hutchinson, 218384326<br>
    Dale Orders, 219106283

# Imported libraries

In [137]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from collections import Counter

<h3> Read in the data, store if dataframe

In [138]:
df = pd.read_csv('./data/healthcare-dataset-stroke-data.csv')

# Tidying up the data

Implementing consistent capitalization and replacing underscores  spaces with hyphens in the data.

In [139]:
##convert gender to lowercase
df['gender'] = df['gender'].apply(lambda x: x.lower())

##convert work_type to lowercase ensure consistent spacing 
df['work_type'] = df['work_type'].apply(lambda x: x.lower().replace('_','-'))

##convert residence_type to lowercase
df.rename(columns={'Residence_type':'residence_type'}, inplace=True)
df['residence_type'] = df['residence_type'].apply(lambda x: x.lower())

##convert smoking_status to lowercase ensure consistent spacing 
df['smoking_status'] = df['smoking_status'].apply(lambda x: x.lower().replace(' ', '-'))

<h3>Replacing gender with dummy variables for easier visualisation

In [140]:
df['gender'] = df['gender'].str.lower().map({'male': 1, 'female': 0})

<h3> Indexes of the outliers with a bmi above 60 for verification purposes

In [141]:
indexes = [270, 358, 466, 544, 928, 1559, 2128, 2764, 2840, 3825, 4188, 4209, 4838]
for index, i in enumerate(df['bmi']):
    if index in indexes:
        print(i, end=',')

60.9,64.8,60.2,71.9,78.0,66.8,97.6,63.3,61.2,61.6,64.4,92.0,60.9,

<h3> Cap the outliers at a maximum bmi of 60

In [142]:
df['bmi'] = df['bmi'].apply(lambda bmi_value: bmi_value if 12 < bmi_value < 60 else np.nan)

<h3>Preliminary work for binning

In [143]:
##max age of 82, max bin get be set accordingly 
print("The max value in age is : " + str(df['age'].max()))
print("Total number of nulls in age is: " + str(df['age'].isnull().sum())) ##check for null
print("Total number of NAs in age is : " + str(df['age'].isna().sum())) ##check for NAs

The max value in age is : 82.0
Total number of nulls in age is: 0
Total number of NAs in age is : 0


<h3>Unknown_smoking status breakdown

In [144]:
##loop to find the number of unknown smoking entries
unknown_count = 0
##store the indexes of the unknown smoking status entries in case it's needed later
unknown_indexes = []
for index, i in enumerate(df['smoking_status']):
    if(i.lower() == 'unknown'):
        #get index and value 
        #print(index, i)
        unknown_indexes.append(index)
        unknown_count+=1
##get percentage of unknown
unknown_percent = unknown_count/len(df) * 100
unknown_percent = round(unknown_percent,4)
print("\nThe percent of entries with an unknown smoking status is: " + str(unknown_percent) + "% or " +str(unknown_count) + " entries")


The percent of entries with an unknown smoking status is: 30.2153% or 1544 entries


<h3>Checking for non-binary values</h3>
<P>Checked in heart_disease, hypertension, ever_married and stroke. Also converts ever_married to 0/1 instead of yes/no<P>

In [145]:
##check for any non-binary values in heart_disease
non_bin_heart_count = 0
for val in df['heart_disease']:
    if(val != 1 and val != 0):
        print(val)
if(non_bin_heart_count == 0):
    print("No non-binary values found in heart_disease")

##check for any non-binary values in hypertension
non_bin_hypt_count = 0
for val in df['hypertension']:
    if(val != 1 and val != 0):
        print(val)
if(non_bin_hypt_count == 0):
    print("No non-binary values found in hypertension")
    
##converts ever_married to 0/1 if not already done 
if(isinstance(df['ever_married'][0],str)):
    df['ever_married'] = df['ever_married'].str.lower().map({'yes': 1, 'no': 0})

##checks for non-binary values in ever_married
non_bin_married_count = 0
for val in df['ever_married']:
    if(val != 1 and val != 0):
        print(val)
if(non_bin_married_count == 0):
    print("No non-binary values found in ever_married")
    
##check for any non-binary values in stroke
non_bin_stroke_count = 0
for val in df['stroke']:
    if(val != 1 and val != 0):
        print(val)
if(non_bin_stroke_count == 0):
    print("No non-binary values found in stroke")

No non-binary values found in heart_disease
No non-binary values found in hypertension
No non-binary values found in ever_married
No non-binary values found in stroke


### Impute missing BMI values
<P>Replace missing BMI values with the average BMI found in rows with the same age and gender<P>

#### Check initial number of nulls

In [146]:
##get number of nulls in df
df.isnull().sum()

id                     0
gender                 1
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
residence_type         0
avg_glucose_level      0
bmi                  218
smoking_status         0
stroke                 0
dtype: int64

<h3>Impute missing values 

In [147]:
df['bmi_raw'] = df['bmi']

df['age'] = df['age'].apply(lambda x : round(x))

m_df = df[df['gender'] == 1]
f_df = df[df['gender'] == 0]

m_bmi_avg = m_df.groupby('age')['bmi'].mean()
f_bmi_avg = f_df.groupby('age')['bmi'].mean()
##round to one to fit with other bmi values
m_bmi_avg = round(m_bmi_avg,1)
f_bmi_avg = round(f_bmi_avg,1)

missing_vals = df[df.isnull().any(axis = 1)]

for index, row in missing_vals.iterrows():
    if row['gender'] == 1:
        df.loc[index,['bmi']] = m_bmi_avg[row['age']]
    else:
        df.loc[index,['bmi']] = f_bmi_avg[row['age']]


<h3>Checking again for nulls to verify imputation success</h3>
<p>NOTE: the values with nulls were left in a separate column in case it is needed later

In [148]:
df.isnull().sum()

id                     0
gender                 1
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
residence_type         0
avg_glucose_level      0
bmi                    0
smoking_status         0
stroke                 0
bmi_raw              218
dtype: int64

<h3>Demonstrating imputed values using the outliers above 60</h2>

In [149]:
indexes = [270, 358, 466, 544, 928, 1559, 2128, 2764, 2840, 3825, 4188, 4209, 4838]
for index, i in enumerate(df['bmi']):
    if index in indexes:
        print(i, end=',')

32.8,33.5,29.1,32.5,27.9,32.3,28.0,26.2,31.5,31.5,27.9,31.8,31.0,

<h2> Equal-width binning implementation</h2>
    <P>Uncomment if you want to use, will not work if equal-size binning has already been implemented
    if you've already applied equal-size, then read in the data-frame again<P>

In [150]:
##define labels and bins using equal width binning
#bins = [0,10,20,30,40,50,60,70,80,90]
#labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90']
##replace all age values with their corresponding bin
#df['age_bin'] = pd.cut(df['age'], bins=bins,labels=labels)

#print("total entires in each bin is:\n", df['age_bin'].value_counts())
#print("\npercentage in each bin is:\n", df['age_bin'].value_counts()/len(df)*100)

<h2> Equal-size binning implementation</h2>
<p>Implements equal-size binning using pd.qcut, binning is performed for age, glucose level and bmi

In [151]:
df['age_bin'] = pd.qcut(df['age'], q = 10, precision=1)
print("total entires in each bin is:\n", df['age_bin'].value_counts())
print("\npercentage in each bin is:\n", df['age_bin'].value_counts()/len(df)*100)

total entires in each bin is:
 (57.0, 65.0]    559
(20.0, 30.0]    545
(-0.1, 11.0]    543
(30.0, 38.0]    529
(38.0, 45.0]    519
(51.0, 57.0]    517
(65.0, 75.0]    500
(11.0, 20.0]    482
(75.0, 82.0]    465
(45.0, 51.0]    451
Name: age_bin, dtype: int64

percentage in each bin is:
 (57.0, 65.0]    10.939335
(20.0, 30.0]    10.665362
(-0.1, 11.0]    10.626223
(30.0, 38.0]    10.352250
(38.0, 45.0]    10.156556
(51.0, 57.0]    10.117417
(65.0, 75.0]     9.784736
(11.0, 20.0]     9.432485
(75.0, 82.0]     9.099804
(45.0, 51.0]     8.825832
Name: age_bin, dtype: float64


<h2>Equal-size binning for average-glucose level

In [152]:
##create 10 bins, print values for review
df['avg_glucose_level_bin'] = pd.qcut(df['avg_glucose_level'], q=10, precision=1)
print("total entires in each bin is:\n", df['avg_glucose_level_bin'].value_counts())
print("\npercentage in each bin is:\n", df['avg_glucose_level_bin'].value_counts()/len(df)*100)

total entires in each bin is:
 (108.5, 124.2]    512
(80.0, 85.6]      512
(65.8, 73.8]      512
(192.2, 271.7]    511
(98.9, 108.5]     511
(91.9, 98.9]      511
(55.0, 65.8]      511
(124.2, 192.2]    510
(85.6, 91.9]      510
(73.8, 80.0]      510
Name: avg_glucose_level_bin, dtype: int64

percentage in each bin is:
 (108.5, 124.2]    10.019569
(80.0, 85.6]      10.019569
(65.8, 73.8]      10.019569
(192.2, 271.7]    10.000000
(98.9, 108.5]     10.000000
(91.9, 98.9]      10.000000
(55.0, 65.8]      10.000000
(124.2, 192.2]     9.980431
(85.6, 91.9]       9.980431
(73.8, 80.0]       9.980431
Name: avg_glucose_level_bin, dtype: float64


<h2>Equal-size binning for bmi

In [153]:
df['bmi_bin'] = pd.qcut(df['bmi'], q=10, precision=1)
print("total entires in each bin is:\n", df['bmi_bin'].value_counts())
print("\npercentage in each bin is:\n", df['bmi_bin'].value_counts()/len(df)*100)

total entires in each bin is:
 (12.200000000000001, 19.8]    522
(28.2, 29.9]                  520
(24.7, 26.6]                  519
(34.2, 38.6]                  516
(29.9, 31.8]                  514
(22.6, 24.7]                  510
(38.6, 59.7]                  506
(26.6, 28.2]                  504
(19.8, 22.6]                  502
(31.8, 34.2]                  497
Name: bmi_bin, dtype: int64

percentage in each bin is:
 (12.200000000000001, 19.8]    10.215264
(28.2, 29.9]                  10.176125
(24.7, 26.6]                  10.156556
(34.2, 38.6]                  10.097847
(29.9, 31.8]                  10.058708
(22.6, 24.7]                   9.980431
(38.6, 59.7]                   9.902153
(26.6, 28.2]                   9.863014
(19.8, 22.6]                   9.823875
(31.8, 34.2]                   9.726027
Name: bmi_bin, dtype: float64


In [154]:
df.isnull().sum()

id                         0
gender                     1
age                        0
hypertension               0
heart_disease              0
ever_married               0
work_type                  0
residence_type             0
avg_glucose_level          0
bmi                        0
smoking_status             0
stroke                     0
bmi_raw                  218
age_bin                    0
avg_glucose_level_bin      0
bmi_bin                    0
dtype: int64

In [155]:
df['gender'] = df['gender'].replace(np.nan, 0)

In [156]:
df.isnull().sum()

id                         0
gender                     0
age                        0
hypertension               0
heart_disease              0
ever_married               0
work_type                  0
residence_type             0
avg_glucose_level          0
bmi                        0
smoking_status             0
stroke                     0
bmi_raw                  218
age_bin                    0
avg_glucose_level_bin      0
bmi_bin                    0
dtype: int64

In [157]:
##I realised that gender had all been mapped to float, so I've fixed that he
df['gender'] = df['gender'].astype(int)

In [158]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_raw,age_bin,avg_glucose_level_bin,bmi_bin
0,9046,1,67,0,1,1,private,urban,228.69,36.6,formerly-smoked,1,36.6,"(65.0, 75.0]","(192.2, 271.7]","(34.2, 38.6]"
1,51676,0,61,0,0,1,self-employed,rural,202.21,29.1,never-smoked,1,NaN,"(57.0, 65.0]","(192.2, 271.7]","(28.2, 29.9]"
2,31112,1,80,0,1,1,private,rural,105.92,32.5,never-smoked,1,32.5,"(75.0, 82.0]","(98.9, 108.5]","(31.8, 34.2]"
3,60182,0,49,0,0,1,private,urban,171.23,34.4,smokes,1,34.4,"(45.0, 51.0]","(124.2, 192.2]","(34.2, 38.6]"
4,1665,0,79,1,0,1,self-employed,rural,174.12,24.0,never-smoked,1,24.0,"(75.0, 82.0]","(124.2, 192.2]","(22.6, 24.7]"


In [159]:
df2 = df.drop(['id','work_type','residence_type','smoking_status','bmi_raw','age_bin','avg_glucose_level_bin','bmi_bin'],axis=1)

In [160]:
df3 = df.drop(['id','work_type','residence_type','smoking_status','bmi_raw','avg_glucose_level','bmi'],axis=1)

In [161]:
df2.head()

,gender,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,stroke
0,1,67,0,1,1,228.69,36.6,1
1,0,61,0,0,1,202.21,29.1,1
2,1,80,0,1,1,105.92,32.5,1
3,0,49,0,0,1,171.23,34.4,1
4,0,79,1,0,1,174.12,24.0,1


In [162]:
df3.head()

,gender,age,hypertension,heart_disease,ever_married,stroke,age_bin,avg_glucose_level_bin,bmi_bin
0,1,67,0,1,1,1,"(65.0, 75.0]","(192.2, 271.7]","(34.2, 38.6]"
1,0,61,0,0,1,1,"(57.0, 65.0]","(192.2, 271.7]","(28.2, 29.9]"
2,1,80,0,1,1,1,"(75.0, 82.0]","(98.9, 108.5]","(31.8, 34.2]"
3,0,49,0,0,1,1,"(45.0, 51.0]","(124.2, 192.2]","(34.2, 38.6]"
4,0,79,1,0,1,1,"(75.0, 82.0]","(124.2, 192.2]","(22.6, 24.7]"


In [163]:
df3.dtypes

gender                      int32
age                         int64
hypertension                int64
heart_disease               int64
ever_married                int64
stroke                      int64
age_bin                  category
avg_glucose_level_bin    category
bmi_bin                  category
dtype: object

## Basic decision tree
Very poor recall and precision

In [164]:
data = df2.drop(['stroke'], axis=1)
target = df['stroke']

X_train, X_test, y_train, y_test = train_test_split(
     data, target, test_size=0.3, random_state=0)

print(y_test.shape)
print(X_test.shape)
clf = tree.DecisionTreeClassifier()
#clf = clf.fit(X_train,y_train)
clf.fit(X_train,y_train)
pred = clf.predict(X_test)

print(classification_report(y_test,pred))

(1533,)
(1533, 7)
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      1457
           1       0.08      0.07      0.07        76

    accuracy                           0.92      1533
   macro avg       0.52      0.51      0.52      1533
weighted avg       0.91      0.92      0.91      1533



In [165]:
confusion_matrix(y_test,pred)

array([[1403,   54],
       [  71,    5]], dtype=int64)

## GBC no SMOTE
Also very poor recall and precision

In [166]:
X_train, X_test, y_train, y_test = train_test_split(
     data, target, test_size=0.3, random_state=0)

gb_clf = GradientBoostingClassifier(loss='deviance',
                                    max_depth=10,
                                    learning_rate=0.5,
                                    n_estimators=600,
                                    random_state=42,
                                    criterion='friedman_mse',
                                    min_samples_split=0.01).fit(X_train, y_train)
print(y_test.shape)
print(X_test.shape)
##gb_clf = gb_clf.fit(X_train,y_train)
gb_clf.fit(X_train,y_train)
pred = gb_clf.predict(X_test)

print(classification_report(y_test,pred))

(1533,)
(1533, 7)
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1457
           1       0.18      0.05      0.08        76

    accuracy                           0.94      1533
   macro avg       0.57      0.52      0.53      1533
weighted avg       0.91      0.94      0.93      1533



In [167]:
len(X_test)

1533

In [168]:
confusion_matrix(y_test,pred)

array([[1439,   18],
       [  72,    4]], dtype=int64)

## Setting up SMOTE

In [169]:
#X_smote, y_smote = SMOTE().fit_resample(data, target)

#counter = Counter(y_smote)
#print('After',counter)

### Use resampled dataset
Results significantly worse, uncomment to test. Generally acc, recall and prec fall by at least 10%

In [170]:
# Find standard deviation of age in stroke victim - take age range of 2 standard deviations in age,
# then compare heart disease, employment, etc.

#stroke_mean_age = df[df['stroke'] == 1].age.mean()
#stroke_age_sd = df[df['stroke'] == 1].age.std()

#print(f'Mean stroke age is {stroke_mean_age:.2f}')
#print(f'New data set will include examples between the ages of {stroke_mean_age - stroke_age_sd * 2:.2f} and {stroke_mean_age + stroke_age_sd * 2:.2f}')

# All examples X where X.age is within 1 SD of median stroke age
# df1sd = df[(df['age'] > stroke_mean_age - stroke_age_sd) & (df['age'] < stroke_mean_age + stroke_age_sd)]

# All examples X where X.age is within 2 SD of median stroke age
#df2sd = df[(df['age'] > stroke_mean_age - stroke_age_sd * 2) & (df['age'] < stroke_mean_age + stroke_age_sd * 2)]
 
# print(f'Dataset counts \nMain data set: {len(df)} \nWithin 1 SD of mean stroke age: {len(df1sd)} \nWithin 2 SD of mean stroke age: {len(df2sd)}')
#print(f'Dataset df2sd contains {len(df2sd)} examples of the main dataset\'s {len(df)}, and {len(df2sd[df2sd.stroke == 1]) / len(df[df.stroke == 1]) * 100:.2f}% of all positive labels')

In [171]:
#df2sd = df2sd.drop(['id','work_type','residence_type','smoking_status','bmi_raw','age_bin','avg_glucose_level_bin','bmi_bin'],axis=1)
#data = df2sd.drop(['stroke'], axis=1)
#target = df2sd['stroke']

In [172]:
#X_smote, y_smote = SMOTE().fit_resample(data, target)

#counter = Counter(y_smote)
#print('After',counter)

## GBC with SMOTE
Much better results, good acc, recall and precision

In [185]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

X_smote, y_smote = SMOTE().fit_resample(X_train, y_train)

counter = Counter(y_smote)
print('After',counter)

gb_clf = GradientBoostingClassifier(loss='deviance',
                                    max_depth=10,
                                    learning_rate=0.5,
                                    n_estimators=600,
                                    random_state=42,
                                    criterion='friedman_mse',
                                    min_samples_split=0.01).fit(X_train, y_train)
gb_clf.fit(X_smote,y_smote)
pred = gb_clf.predict(X_test)

print(classification_report(y_test,pred))

After Counter({0: 3403, 1: 3403})
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1458
           1       0.15      0.21      0.17        75

    accuracy                           0.90      1533
   macro avg       0.55      0.58      0.56      1533
weighted avg       0.92      0.90      0.91      1533



In [186]:
confusion_matrix(y_test,pred)

array([[1366,   92],
       [  59,   16]], dtype=int64)

In [175]:
len(X_test)

1533

In [176]:
len(X_train)

3577

In [177]:
len(y_test)

1533

In [178]:
len(y_train)

3577

In [179]:
len(X_smote)

6812

In [180]:
len(y_smote)

6812

In [181]:
X_smote

,gender,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi
0,0,45,0,0,1,84.990000,35.400000
1,1,36,0,0,1,129.730000,27.800000
2,1,49,1,0,1,139.430000,40.200000
3,0,59,1,0,1,92.040000,24.200000
4,0,18,0,0,0,98.100000,21.800000
...,...,...,...,...,...,...,...
6807,0,72,0,0,1,98.049263,26.633267
6808,0,72,0,0,0,71.064921,34.643748
6809,0,56,0,0,1,189.407907,38.335289
6810,0,62,0,0,1,198.057730,30.808564


In [182]:
y_smote

0       0
1       0
2       0
3       0
4       0
       ..
6807    1
6808    1
6809    1
6810    1
6811    1
Name: stroke, Length: 6812, dtype: int64